In [1]:
import pandas as pd

import sys

import pandas as pd
import time
import numpy as np
from nba_api.stats.endpoints import playercareerstats
import nba_api.stats.endpoints as nbaapi

pd.set_option('display.max_columns', 500)

In [5]:
from nba_api.stats.static import players
from nba_api.stats.static import teams

# get_players returns a list of dictionaries, each representing a player.
nba_players = players.get_players()
nba_teams = teams.get_teams()

nba_players = pd.DataFrame(nba_players)
nba_teams = pd.DataFrame(nba_teams)

active_plyaers = nba_players[nba_players['is_active']]

seasons = []
for i in list(range(1983,2024)):
    seasons.append(str(i)+'-'+str(i+1)[2:])
seasons = seasons[::-1]
    
### 
testplayer='203076'
testgame = '0022201200'
testteam = '1610612755'

In [118]:
from nba_api.stats.endpoints import leaguegamefinder

result = leaguegamefinder.LeagueGameFinder()
all_games = result.get_data_frames()[0]

listframe = []
for i in seasons[:1]:
    playerfinder = nbaapi.leaguedashplayerbiostats.LeagueDashPlayerBioStats(season=i).get_data_frames()[0]
    playerfinder['season']=i
    listframe.append(playerfinder)
    time.sleep(.5)
allplayers = pd.concat(listframe,axis=0)

listframe = []
for i in seasons[:1]:
    gamefinder = leaguegamefinder.LeagueGameFinder(league_id_nullable='00', season_type_nullable="Regular Season",season_nullable=i).get_data_frames()[0]
    listframe.append(gamefinder)
    time.sleep(1)
gamehistory = pd.concat(listframe,axis=0)
#gamehistory['GAME_ID'] = pd.to_numeric(gamehistory['GAME_ID'])
gamehistory['GAME_DATE'] = pd.to_datetime(gamehistory['GAME_DATE'])

In [128]:
inseasondates = ['2023-11-03','2023-12-04','2023-12-05']
gamehistory.loc[gamehistory['GAME_DATE'].isin(inseasondates),'INSEASON']=1
gamehistory.loc[(gamehistory['GAME_DATE'].dt.day.between(3,29)) & (gamehistory['GAME_DATE'].dt.month == 11) & (gamehistory['GAME_DATE'].dt.dayofweek.isin([1,4])),'INSEASON'] = 1
#gamehistory[gamehistory['INSEASON']==1].to_csv('test.csv')

isgames = gamehistory[gamehistory['INSEASON']==1].GAME_ID.drop_duplicates()
hometeam = gamehistory[gamehistory['MATCHUP'].str.contains('vs')][['GAME_ID','TEAM_ID']].rename(columns={'TEAM_ID':'HOME_TEAM'})
courtcolors = pd.read_csv('team_court_colors.csv')

In [129]:
# listframe = []
# for i in gamehistory['GAME_ID'].astype(str):#.drop_duplicates():
#     tempdf = nbaapi.boxscoretraditionalv2.BoxScoreTraditionalV2(game_id='00'+i).get_data_frames()[0]#[['GAME_ID','TEAM_ID','TEAM_ABBREVIATION','PLAYER_ID','PLAYER_NAME','MIN','SPD','DIST']]
#     listframe.append(tempdf)
#     time.sleep(3)
# aggdf_player = pd.concat(listframe,axis=0)
# aggdf_player.to_csv('./data/game_player.csv',index=None)


In [156]:
df1 = aggdf_player.merge(hometeam,on='GAME_ID')
df2 = df1.merge(courtcolors[['TEAM_ID','Major_Color','Minor_Color']],left_on='HOME_TEAM',right_on='TEAM_ID').drop(columns='TEAM_ID_y').rename(columns={'TEAM_ID_x':'TEAM_ID'})
df3 = df2.merge(gamehistory[['GAME_ID','GAME_DATE']],on='GAME_ID')

df3.loc[df3['GAME_ID'].isin(isgames),'inseason_tourney_game'] = True
df3.loc[df3['inseason_tourney_game'].isnull(),'Major_Color'] = 'FDDCB9'
df3.loc[df3['inseason_tourney_game'].isnull(),'Minor_Color'] = 'FDDCB9'

df3['MajRed'] = df3['Major_Color'].map(lambda x: x[0:2]).apply(lambda x: int(x, 16))
df3['MajGreen'] = df3['Major_Color'].map(lambda x: x[2:4]).apply(lambda x: int(x, 16))
df3['MajBlue'] = df3['Major_Color'].map(lambda x: x[4:6]).apply(lambda x: int(x, 16))
df3['MinRed'] = df3['Minor_Color'].map(lambda x: x[0:2]).apply(lambda x: int(x, 16))
df3['MinGreen'] = df3['Minor_Color'].map(lambda x: x[2:4]).apply(lambda x: int(x, 16))
df3['MinBlue'] = df3['Minor_Color'].map(lambda x: x[4:6]).apply(lambda x: int(x, 16))

df3.drop_duplicates().to_csv('./data/master_data.csv',index=None)

In [1]:
df3

NameError: name 'df3' is not defined

In [158]:
df3.drop_duplicates()

,GAME_ID,TEAM_ID,TEAM_ABBREVIATION,TEAM_CITY,PLAYER_ID,PLAYER_NAME,NICKNAME,START_POSITION,COMMENT,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TO,PF,PTS,PLUS_MINUS,HOME_TEAM,Major_Color,Minor_Color,GAME_DATE,inseason_tourney_game,MajRed,MajGreen,MajBlue,MinRed,MinGreen,MinBlue
0,0022301210,1610612759,SAS,San Antonio,1626224,Cedi Osman,Cedi,F,,32.000000:09,3.0,8.0,0.375,0.0,3.0,0.0,0.0,2.0,0.000,2.0,6.0,8.0,6.0,2.0,0.0,1.0,2.0,6.0,-10.0,1610612750,FDDCB9,FDDCB9,2023-12-06,NaN,253,220,185,253,220,185
2,0022301210,1610612759,SAS,San Antonio,1641705,Victor Wembanyama,Victor,F,,32.000000:25,4.0,13.0,0.308,0.0,2.0,0.0,4.0,6.0,0.667,3.0,7.0,10.0,1.0,1.0,1.0,2.0,1.0,12.0,4.0,1610612750,FDDCB9,FDDCB9,2023-12-06,NaN,253,220,185,253,220,185
4,0022301210,1610612759,SAS,San Antonio,1628380,Zach Collins,Zach,C,,30.000000:16,6.0,20.0,0.300,0.0,6.0,0.0,0.0,0.0,0.000,4.0,4.0,8.0,4.0,0.0,0.0,3.0,3.0,12.0,-17.0,1610612750,FDDCB9,FDDCB9,2023-12-06,NaN,253,220,185,253,220,185
6,0022301210,1610612759,SAS,San Antonio,1629640,Keldon Johnson,Keldon,G,,31.000000:51,8.0,15.0,0.533,1.0,2.0,0.5,4.0,5.0,0.800,1.0,9.0,10.0,3.0,2.0,1.0,4.0,4.0,21.0,-3.0,1610612750,FDDCB9,FDDCB9,2023-12-06,NaN,253,220,185,253,220,185
8,0022301210,1610612759,SAS,San Antonio,1630170,Devin Vassell,Devin,G,,31.000000:17,9.0,16.0,0.563,4.0,8.0,0.5,0.0,0.0,0.000,0.0,3.0,3.0,2.0,1.0,1.0,0.0,1.0,22.0,4.0,1610612750,FDDCB9,FDDCB9,2023-12-06,NaN,253,220,185,253,220,185
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32698,0022300086,1610612757,POR,Portland,1631133,Jabari Walker,Jabari,,,9.000000:23,2.0,3.0,0.667,1.0,2.0,0.5,4.0,4.0,1.000,1.0,3.0,4.0,0.0,0.0,0.0,0.0,3.0,9.0,1.0,1610612757,FDDCB9,FDDCB9,2023-10-27,NaN,253,220,185,253,220,185
32700,0022300086,1610612757,POR,Portland,1630219,Skylar Mays,Skylar,,,7.000000:32,1.0,2.0,0.500,0.0,0.0,0.0,0.0,0.0,0.000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,2.0,0.0,1610612757,FDDCB9,FDDCB9,2023-10-27,NaN,253,220,185,253,220,185
32702,0022300086,1610612757,POR,Portland,1629650,Moses Brown,Moses,,DNP - Coach's Decision,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612757,FDDCB9,FDDCB9,2023-10-27,NaN,253,220,185,253,220,185
32704,0022300086,1610612757,POR,Portland,1631200,Kris Murray,Kris,,DNP - Coach's Decision,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1610612757,FDDCB9,FDDCB9,2023-10-27,NaN,253,220,185,253,220,185
